In [1]:
import osmnx as ox
import networkx as nx
import matplotlib.cm as cm
import matplotlib.colors as colors
import pandas as pd
import datetime
import geocoder
import geopandas as gpd
from glob import glob
import os
%matplotlib inline
ox.config(log_console=True, use_cache=True)

In [2]:
def get_time(t1, t2):
    
    diff = t2 - t1
    
    c = round(diff.total_seconds() / 60, 2)
    
    return c

In [3]:
def get_graph_old(place):
    string = place.split(',')[0]
    
    print('Fetching graph data for {}'.format(place))
    # download and project a street network
    if not place == 'Monrovia, Liberia':
        try:
            G = ox.graph_from_place(place, network_type='drive')
        except TypeError:
            try:
                G = ox.graph_from_place(place, network_type='drive', which_result=2)
            except TypeError:
                geo = geocoder.osm(place)
                lon = geo.geojson['features'][0]['properties']['lng']
                lat = geo.geojson['features'][0]['properties']['lng']
                G = ox.graph_from_point(center_point=( lon, lat), distance = 20000)
            except KeyError:
                geo = geocoder.osm(place)
                lon = geo.geojson['features'][0]['properties']['lng']
                lat = geo.geojson['features'][0]['properties']['lng']
                G = ox.graph_from_point(center_point=( lon, lat), distance = 20000)
        except:
            geo = geocoder.osm(place)
            lon = geo.geojson['features'][0]['properties']['lng']
            lat = geo.geojson['features'][0]['properties']['lng']
            G = ox.graph_from_point(center_point=( lon, lat), distance = 20000)
    else:
        adm3 = gpd.GeoDataFrame.from_file('Liberia/LBR_adm3.shp')
        mnv = adm3[adm3.NAME_3 == 'Monrovia'].geometry.values[0]
        G = ox.graph_from_polygon(mnv, network_type='drive')
        
    G = ox.project_graph(G)
    
    print('Writing graph file')
    
    try:
        os.mkdir('data/{}'.format(string))
    except FileExistsError:
        pass
    
    nx.write_gpickle(G, path='data/{a}/{a}'.format(a=string))
    
    return G

In [4]:
def get_polygon(string):
    
    lis = glob('/nfshome/gb1877/gb1877/WorldBank/CRP/Betweenness_Centrality/data/AOI/*/*.shp')
    
    city = []
    for i in lis:
        x = i.split('/')[-2]
        city.append(x.split('_')[0])
        
    if string in city:
        l = glob('/nfshome/gb1877/gb1877/WorldBank/CRP/Betweenness_Centrality/data/AOI/{}_AOI/*.shp'.format(string))
        adm = gpd.GeoDataFrame.from_file(l[0])
        boundary_poly = adm.geometry.values[0]
    else:
        boundary_GDF = ox.gdf_from_place('{}'.format(string),which_result=1)
        boundary_poly = boundary_GDF.loc[0,'geometry']
        if boundary_poly.geom_type == 'Polygon':
            boundary_poly = boundary_poly
        else:
            try:
                boundary_GDF = ox.gdf_from_place('{}'.format(string),which_result=2)
                boundary_poly = boundary_GDF.loc[0,'geometry']
            except:
                print('Polygon not available')
    
    return boundary_poly

In [5]:
def get_graph(place):
    string = place.split(',')[0]
    
    print('Fetching graph data for {}'.format(place))

    G = ox.graph_from_polygon(get_polygon(string), network_type='drive')
    
    G = ox.project_graph(G)
    
    print('Writing graph file')
    
    try:
        os.mkdir('data/{}'.format(string))
    except FileExistsError:
        pass
    
    nx.write_gpickle(G, path='data/{a}/{a}'.format(a=string))
    
    return G

In [6]:
def get_edge_centrality(place):
    
    t1 = datetime.datetime.now()
    
    string = place.split(',')[0]
    
    # download and project a street network
    
    G = get_graph(place)
    
    t2 = datetime.datetime.now()
    
    print('{} minutes elapsed!'.format(get_time(t1, t2)))
    
    print('Getting node centrality')
    node_centrality = nx.betweenness_centrality(G)
    
    
    t3 = datetime.datetime.now()
    
    print('{} minutes elapsed!'.format(get_time(t1, t3)))
    
    print('Getting edge centrality')
    # edge closeness centrality: convert graph to a line graph so edges become nodes and vice versa
    edge_centrality = nx.betweenness_centrality(nx.line_graph(G))
    
    t4 = datetime.datetime.now()
    
    print('{} minutes elapsed!'.format(get_time(t1, t4)))
    
    print('Saving output gdf')
    
    nx.set_node_attributes(G, node_centrality, 'node_centrality')
    nx.set_edge_attributes(G, edge_centrality, 'edge_centrality')
    ox.save_graph_shapefile(G, filename='{}'.format(string))
    
    t5 = datetime.datetime.now()
    
    print('{} minutes elapsed!'.format(get_time(t1, t5)))
    
    print('Getting extended stats')
    
    extended_stats = ox.extended_stats(G, bc=True)
    
    dat = pd.DataFrame.from_dict(extended_stats)
    dat.to_csv('data/{a}/Extended_Stats_{b}.csv'.format(a=string, b=string))
    
    t6 = datetime.datetime.now()
    
    print('{} minutes elapsed!'.format(get_time(t1, t6)))
    
    print('Getting basic stats')
    
    basic_stats = ox.basic_stats(G)
    dat = pd.DataFrame.from_dict(basic_stats)
    dat.to_csv('data/{a}/Basic_Stats_{b}.csv'.format(a=string, b=string))
    
    return

1. Sfax, Tunisia
2. Sousse, Tunisia
3. Tunis, Tunisia
4. Batroun, Lebanon
5. Bint Jbeil, Lebanon
6. Joub Jannine, Lebanon
7. West Bank
8. Montevideo, Uruguay
9. Lima, Peru
10. Cap-Haitien, Haiti
11. Vinh Long, Vietnam
12. Khan Hoa, Vietnam
13. Semarang, Indonesia
14. Penang, Malaysia
15. Tashkent, Uzbekistan
16. Osh, Kyrgyz 
17. Turkey (city TBD)
18. Monrovia, Liberia 
19. Johannesburg, South Africa
20. Maputo, Mozambique 
21. Addis Ababa, Ethiopia 
22. Blantyr, Malawi
23. St. Louis, Senegal (TBC)
24. Cotonou, Benin (TBC)
25. Accra, Ghana (TBC)
26. Province of Palawan, Philippines
27. Davao City, Philippines

In [6]:
city_scan_list = [
    'Sfax, Tunisia',
'Sousse, Tunisia',
'Tunis, Tunisia',
'Batroun, Lebanon',
'Bint Jbeil, Lebanon',
'Joub Jannine, Lebanon',
#'West Bank', 
'Montevideo, Uruguay',
'Lima, Peru',
'Cap-Haitien, Haiti',
'Vinh Long, Vietnam',
'Khanh Hoa, Vietnam',
'Semarang, Indonesia',
'Penang, Malaysia',
'Tashkent, Uzbekistan',
'Osh,  Kyrgyzstan',
#Turkey (city TBD)
'Monrovia, Liberia',
'Johannesburg, South Africa',
'Maputo, Mozambique',
'Addis Ababa, Ethiopia',
'Blantyr, Malawi',
'St. Louis, Senegal',
'Cotonou, Benin',
'Accra, Ghana',
'Province of Palawan, Philippines',
'Davao City, Philippines'
    
]

In [ ]:
for i in city_scan_list[13:]:
    %time get_edge_centrality(i)

Fetching graph data for Tashkent, Uzbekistan
Writing graph file
1.23 minutes elapsed!
Getting node centrality
25.49 minutes elapsed!
Getting edge centrality


In [5]:
problem_cities = ['Bint Jbeil, Lebanon',
'Joub Jannine, Lebanon','Khanh Hoa, Vietnam','Semarang, Indonesia',
'Penang, Malaysia','Monrovia, Liberia','Johannesburg, South Africa',
                 'Addis Ababa, Ethiopia','Blantyr, Malawi','St. Louis, Senegal',
                  'Cotonou, Benin', 'Accra, Ghana',]

In [10]:
%time get_edge_centrality('Joub Jannine, Lebanon')

Fetching graph data for Joub Jannine, Lebanon
Writing graph file
0.65 minutes elapsed!
Getting node centrality
1.05 minutes elapsed!
Getting edge centrality
4.57 minutes elapsed!
Saving output gdf
4.68 minutes elapsed!
Getting extended stats
5.57 minutes elapsed!
Getting basic stats
CPU times: user 5min 27s, sys: 940 ms, total: 5min 27s
Wall time: 5min 34s


In [12]:
%time get_edge_centrality('Lima, Peru')

Fetching graph data for Lima, Peru
Writing graph file
1.04 minutes elapsed!
Getting node centrality
1.53 minutes elapsed!
Getting edge centrality
4.96 minutes elapsed!
Saving output gdf
5.08 minutes elapsed!
Getting extended stats
6.23 minutes elapsed!
Getting basic stats
CPU times: user 5min 24s, sys: 1.08 s, total: 5min 25s
Wall time: 6min 14s


In [22]:
%time get_edge_centrality('Monrovia, Liberia')

Fetching graph data for Monrovia, Liberia
Writing graph file
0.93 minutes elapsed!
Getting node centrality
4.27 minutes elapsed!
Getting edge centrality
30.46 minutes elapsed!
Saving output gdf


/nfshome/gb1877/.conda/envs/osmpython/lib/python3.6/site-packages/geopandas/io/file.py:108: FionaDeprecationWarning: Use fiona.Env() instead.
  with fiona.drivers():


30.75 minutes elapsed!
Getting extended stats
37.97 minutes elapsed!
Getting basic stats
CPU times: user 37min 35s, sys: 932 ms, total: 37min 36s
Wall time: 38min 2s


In [ ]:
%time get_edge_centrality('West Bank')

Fetching graph data for West Bank
Writing graph file
42.15 minutes elapsed!
Getting node centrality


### Priority cities

- El Nido, Philippines
- Coron, Philippines 
- Puerto Princesa, Philippines
- Davao City, Philippines
- Tashkent, Uzbekistan
- West Bank
- Lima, Peru
- Vinh Long, Vietnam
- Khan Hoa, Vietnam
- Osh, Kyrgyz
- Monrovia, Liberia


In [31]:
city

['Puerto Princesa',
 'West Bank',
 'Davao',
 'Coron',
 'Khan Hoa',
 'Bethlehem',
 'Monrovia',
 'Hebron',
 'Osh',
 'El Nido',
 'Lima',
 'Tashkent',
 'Penang',
 'Vinh Long']

In [ ]:
for i in city:
    %time get_edge_centrality(i)

Fetching graph data for Puerto Princesa
Writing graph file
0.32 minutes elapsed!
Getting node centrality
1.33 minutes elapsed!
Getting edge centrality
10.92 minutes elapsed!
Saving output gdf


/nfshome/gb1877/.conda/envs/osmpython/lib/python3.6/site-packages/geopandas/io/file.py:108: FionaDeprecationWarning: Use fiona.Env() instead.
  with fiona.drivers():


11.07 minutes elapsed!
Getting extended stats
13.36 minutes elapsed!
Getting basic stats
CPU times: user 13min 14s, sys: 720 ms, total: 13min 15s
Wall time: 13min 21s
Fetching graph data for West Bank
Writing graph file
0.93 minutes elapsed!
Getting node centrality
8.34 minutes elapsed!
Getting edge centrality
86.47 minutes elapsed!
Saving output gdf


86.86 minutes elapsed!
Getting extended stats
103.25 minutes elapsed!
Getting basic stats
CPU times: user 1h 43min 12s, sys: 1.96 s, total: 1h 43min 14s
Wall time: 1h 43min 16s
Fetching graph data for Davao
Writing graph file
0.54 minutes elapsed!
Getting node centrality
3.88 minutes elapsed!
Getting edge centrality
36.33 minutes elapsed!
Saving output gdf


36.59 minutes elapsed!
Getting extended stats
43.97 minutes elapsed!
Getting basic stats
CPU times: user 43min 55s, sys: 1.16 s, total: 43min 56s
Wall time: 43min 58s
Fetching graph data for Coron
Writing graph file
0.03 minutes elapsed!
Getting node centrality
0.03 minutes elapsed!
Getting edge centrality


0.03 minutes elapsed!
Saving output gdf


0.04 minutes elapsed!
Getting extended stats
0.04 minutes elapsed!
Getting basic stats
CPU times: user 1.51 s, sys: 4 ms, total: 1.51 s
Wall time: 2.53 s
Fetching graph data for Khan Hoa
Writing graph file
1.07 minutes elapsed!
Getting node centrality
5.12 minutes elapsed!
Getting edge centrality
44.68 minutes elapsed!
Saving output gdf


45.0 minutes elapsed!
Getting extended stats
54.33 minutes elapsed!
Getting basic stats
CPU times: user 54min 10s, sys: 1.69 s, total: 54min 12s
Wall time: 54min 21s
Fetching graph data for Bethlehem
Writing graph file
0.91 minutes elapsed!
Getting node centrality
2.25 minutes elapsed!
Getting edge centrality
10.65 minutes elapsed!
Saving output gdf


10.81 minutes elapsed!
Getting extended stats
13.37 minutes elapsed!
Getting basic stats
CPU times: user 13min 21s, sys: 1.06 s, total: 13min 22s
Wall time: 13min 23s
Fetching graph data for Monrovia
Writing graph file
1.05 minutes elapsed!
Getting node centrality
8.29 minutes elapsed!
Getting edge centrality
59.8 minutes elapsed!
Saving output gdf


60.12 minutes elapsed!
Getting extended stats
73.85 minutes elapsed!
Getting basic stats
CPU times: user 1h 13min 49s, sys: 1.98 s, total: 1h 13min 51s
Wall time: 1h 13min 52s
Fetching graph data for Hebron
Writing graph file
4.36 minutes elapsed!
Getting node centrality


In [8]:
%time get_edge_centrality('Tashkent, Uzbekistan')

Fetching graph data for Tashkent, Uzbekistan
Writing graph file
1.33 minutes elapsed!
Getting node centrality
25.43 minutes elapsed!
Getting edge centrality
235.26 minutes elapsed!
Saving output gdf


/nfshome/gb1877/.conda/envs/osmpython/lib/python3.6/site-packages/geopandas/io/file.py:108: FionaDeprecationWarning: Use fiona.Env() instead.
  with fiona.drivers():


235.99 minutes elapsed!
Getting extended stats
292.0 minutes elapsed!
Getting basic stats
CPU times: user 4h 51min 34s, sys: 11.6 s, total: 4h 51min 45s
Wall time: 4h 52min 2s


In [9]:
%time get_edge_centrality('Vinh Long, Vietnam')

Fetching graph data for Vinh Long, Vietnam
Writing graph file
0.06 minutes elapsed!
Getting node centrality
0.09 minutes elapsed!
Getting edge centrality
0.41 minutes elapsed!
Saving output gdf
0.45 minutes elapsed!
Getting extended stats
0.54 minutes elapsed!
Getting basic stats
CPU times: user 30.8 s, sys: 144 ms, total: 30.9 s
Wall time: 32.2 s
